In [1]:
!pip install kfp==1.8.4

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.2/252.2 kB 34.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.8/127.8 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 kB 19.8 MB/s eta 0:00:00
  Created wheel for kfp: filename=kfp-1.8.4-py3-none-any.whl size=349008 sha256=a993f281b982b48e7cc073b470be0cab54c8f0bb51c88c4ea55b76eac9c22e88
  Stored in directory: /home/jovyan/.cache/pip/wheels/49/2b/fc/79b1a2787e4f73f9ee6b1c7fe88f7f42b82133c40baca38aa9
Successfully built kfp
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
import kfp
from kfp import components

from kfp import dsl
from kfp import compiler

from kubeflow.training.utils import utils


In [4]:
pytorch_job_op = components.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/master/components/kubeflow/pytorch-launcher/component.yaml')

In [5]:
kanwal_namespace = utils.get_default_target_namespace()
kanwal_namespace

'kubeflow-user-example-com'

In [6]:
@dsl.pipeline(name="PyTorch Training pipeline", description="Sample training job test")
def pytorch_cnn_n_b_yaml(
    namespace=kanwal_namespace
):
    
    train_task = pytorch_job_op(
        name='pytorch-cnn-dist-file-c3', 
        namespace='kubeflow-user-example-com', 
        master_spec='{ \
          "replicas": 1, \
          "restartPolicy": "OnFailure", \
          "template": { \
            "metadata": { \
              "annotations": { \
                "sidecar.istio.io/inject": "false" \
              } \
            }, \
            "spec": { \
              "containers": [ \
                { \
                  "name": "pytorch1", \
                  "image": "763104351884.dkr.ecr.us-west-2.amazonaws.com/pytorch-training:1.12.0-gpu-py38-cu116-ubuntu20.04-e3", \
                  "args": [ \
                    "python", \
                    "./efs-shared/pipeline/mnist.py", \
                    "--epochs", "5", \
                    "--seed", "7", \
                    "--log-interval", "60" \
                  ], \
                  "resources": { \
                    "limits": { \
                      "nvidia.com/gpu": 1 \
                    } \
                  }, \
                  "volumeMounts": [ \
                    { \
                      "mountPath": "/efs-shared", \
                      "name": "efs-shared" \
                    } \
                  ] \
                } \
              ], \
              "volumes": [ \
                { \
                  "name": "efs-shared", \
                  "persistentVolumeClaim": { \
                    "claimName": "efs-pvc" \
                  } \
                } \
              ] \
            } \
          } \
        }', 
        worker_spec='{ \
          "replicas": 1, \
          "restartPolicy": "OnFailure", \
          "template": { \
            "metadata": { \
              "annotations": { \
                "sidecar.istio.io/inject": "false" \
              } \
            }, \
            "spec": { \
              "containers": [ \
                { \
                  "name": "pytorch2", \
                  "image": "763104351884.dkr.ecr.us-west-2.amazonaws.com/pytorch-training:1.12.0-gpu-py38-cu116-ubuntu20.04-e3", \
                  "args": [ \
                    "python", \
                    "./efs-shared/pipeline/mnist.py", \
                    "--epochs", "5", \
                    "--seed", "7", \
                    "--log-interval", "60" \
                  ], \
                  "resources": { \
                    "limits": { \
                      "nvidia.com/gpu": 1 \
                    } \
                  }, \
                  "volumeMounts": [ \
                    { \
                      "mountPath": "/efs-shared", \
                      "name": "efs-shared" \
                    } \
                  ] \
                } \
              ], \
              "volumes": [ \
                { \
                  "name": "efs-shared", \
                  "persistentVolumeClaim": { \
                    "claimName": "efs-pvc" \
                  } \
                } \
              ] \
            } \
          } \
        }',
        delete_after_done=False
    )
    

In [7]:
kfp.compiler.Compiler().compile(pytorch_cnn_n_b_yaml, "pytorch_cnn_n_b_yaml.zip")